In [0]:
# Load employee.csv file data
df_employee = spark.read.csv('/FileStore/employee_data.csv', header=True, inferSchema=True).cache()
df_employee.show()
df_employee.printSchema()

# Load product_data.json file
df = spark.read.option("multiline", "true").json("/FileStore/product_data.json")
df.show(10)
df.printSchema()

+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 65000|
|      1008|Barbara Moore|        HR| 2021-03-29| 53000|
+----------+-------------+----------+-----------+------+

root
 |-- EmployeeID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JoiningDate: date (nullable = true)
 |-- Salary: integer (nullable = true)

+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------

In [0]:
# 2. Convert CSV and JSON Data to Delta Format
df_employee.write.format("delta").mode("overwrite").save("/dbfs/FileStore/delta/employee_data")
df.write.format("delta").mode("overwrite").save("/dbfs/FileStore/delta/product_data")

In [0]:
# 3. Register Delta Tables as SQL Tables
spark.sql("CREATE TABLE IF NOT EXISTS employee_delta USING DELTA LOCATION '/dbfs/FileStore/delta/employee_data'")
spark.sql("CREATE TABLE IF NOT EXISTS product_delta USING DELTA LOCATION '/dbfs/FileStore/delta/product_data'")

DataFrame[]

In [0]:
# 3. Data Modifications with Delta Tables
# Increase salary by 5% for IT department employees
spark.sql("UPDATE employee_delta SET Salary = Salary * 1.05 WHERE Department = 'IT'")
# Delete products where stock is less than 40
spark.sql("DELETE FROM product_delta WHERE Stock < 40")

DataFrame[num_affected_rows: bigint]

In [0]:
# 4. Time Travel with Delta Tables:
# Query the product Delta table to show its state before the delete
# operation (use time travel).
df_product_version_before_delete = spark.sql("SELECT * FROM product_delta VERSION AS OF 0")
df_product_version_before_delete.show()
# Retrieve the version of the employee Delta table before the salary update.
df_employee_version_before_update = spark.sql("SELECT * FROM employee_delta VERSION AS OF 0")
df_employee_version_before_update.show()

+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics| 1200|      101|     Laptop|   35|
|Electronics|  800|      102| Smartphone|   80|
|  Furniture|  150|      103| Desk Chair|   60|
|Electronics|  300|      104|    Monitor|   45|
|  Furniture|  350|      105|       Desk|   25|
+-----------+-----+---------+-----------+-----+

+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1001|     John Doe|        HR| 2021-01-15| 55000|
|      1002|   Jane Smith|        IT| 2020-03-10| 62000|
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1004|Michael Brown|        HR| 2018-12-22| 54000|
|      1005| David Wilson|        IT| 2021-06-25| 58000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
|      1007| James Miller|        IT| 2019-08-14| 6500

In [0]:
# 5. Query Delta Tables:
# Query the employee Delta table to find the employees in the Finance department.
df_finance_employees = spark.sql("SELECT * FROM employee_delta WHERE Department = 'Finance'")
df_finance_employees.show()
# Query the product Delta table to find all products in the Electronics category with a price greater than 500.
df_expensive_electronics = spark.sql("SELECT * FROM product_delta WHERE Category = 'Electronics' AND Price > 500")
df_expensive_electronics.show()

+----------+-------------+----------+-----------+------+
|EmployeeID|         Name|Department|JoiningDate|Salary|
+----------+-------------+----------+-----------+------+
|      1003|Emily Johnson|   Finance| 2019-07-01| 70000|
|      1006|  Linda Davis|   Finance| 2020-11-15| 67000|
+----------+-------------+----------+-----------+------+

+-----------+-----+---------+-----------+-----+
|   Category|Price|ProductID|ProductName|Stock|
+-----------+-----+---------+-----------+-----+
|Electronics|  800|      102| Smartphone|   80|
+-----------+-----+---------+-----------+-----+

